<a href="https://colab.research.google.com/github/Volt1977/Volt1977/blob/main/%D0%A0%D0%B5%D0%BA%D0%BE%D0%BC%D0%B5%D0%BD%D0%B4%D0%B0%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D0%B0%D1%8F_%D1%81%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Нужно создать систему  рекомендации на основе предоставленных данных, на выходе пакет перелет-отель-экскурсия

In [176]:
!pip install zstandard jsonlines
import pandas as pd
import zstandard as zstd
import json

# Открываем файл и создаем декомпрессор
with open('feed_ru.json.zst', 'rb') as file:
    dctx = zstd.ZstdDecompressor()
    decompressed = dctx.decompressobj()

    # Читаем и декомпрессируем содержимое файла
    content = file.read()
    decompressed_data = decompressed.decompress(content)

    # Преобразуем декомпрессированные данные в строку
    json_string = decompressed_data.decode()

# Загружаем данные в DataFrame
df = pd.read_json(json_string, lines=True)



# Выводим первые несколько строк DataFrame
print(df.head())


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
                                           address       check_in_time  \
0             Via francesco zizzo 28, Санта-Флавия 2023-05-24 23:00:00   
1      Via Cristoforo Colombo; 4, Марина-ди-Модика 2023-05-24 10:00:00   
2  Strada Consortile SantAnna Pollina 131, Альгеро 2023-05-24 20:00:00   
3                      Disseminat 5, 183, Алькудия 2023-05-24 14:00:00   
4         Via Lungomare Luigi Rizzo 131, Лампедуза 2023-05-24 12:00:00   

       check_out_time                                 description_struct  \
0 2023-05-24 11:00:00  [{'paragraphs': ['Прекрасный вариант, чтобы от...   
1 2023-05-24 12:30:00  [{'paragraphs': ['Выбор туристов, путешествующ...   
2 2023-05-24 10:00:00  [{'paragraphs': ['Залог хорошего отдыха — чувс...   
3 2023-05-24 12:00:00  [{'paragraphs': ['Хорошо, когда красота начина...   
4 2023-05-24 10:00:00  [{'paragraphs': ['В поисках домашнего уюта: ап...   


In [177]:
print(len(df))


23112


In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23112 entries, 0 to 23111
Data columns (total 29 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   address                18793 non-null  object        
 1   check_in_time          18793 non-null  datetime64[ns]
 2   check_out_time         18793 non-null  datetime64[ns]
 3   description_struct     18782 non-null  object        
 4   id                     23112 non-null  object        
 5   kind                   18793 non-null  object        
 6   latitude               18793 non-null  float64       
 7   longitude              18793 non-null  float64       
 8   name                   18793 non-null  object        
 9   postal_code            17309 non-null  object        
 10  room_groups            11413 non-null  object        
 11  region                 18793 non-null  object        
 12  deleted                23112 non-null  bool          
 13  i

In [179]:
missing_values = df.isnull().sum()
print(missing_values)


address                   4319
check_in_time             4319
check_out_time            4319
description_struct        4330
id                           0
kind                      4319
latitude                  4319
longitude                 4319
name                      4319
postal_code               5803
room_groups              11699
region                    4319
deleted                      0
is_closed                    0
metapolicy_struct         4319
metapolicy_extra_info    10120
star_certificate         23057
facts                     4319
payment_methods           4319
hotel_chain               4319
front_desk_time_start    23096
front_desk_time_end      23028
serp_filters              9634
star_rating              18371
amenity_groups           10606
email                    16875
images                   14352
phone                    16603
policy_struct            15158
dtype: int64


In [180]:
missing_percentages = df.isnull().sum() / len(df) * 100
print(missing_percentages)




address                  18.687262
check_in_time            18.687262
check_out_time           18.687262
description_struct       18.734856
id                        0.000000
kind                     18.687262
latitude                 18.687262
longitude                18.687262
name                     18.687262
postal_code              25.108169
room_groups              50.618726
region                   18.687262
deleted                   0.000000
is_closed                 0.000000
metapolicy_struct        18.687262
metapolicy_extra_info    43.786777
star_certificate         99.762028
facts                    18.687262
payment_methods          18.687262
hotel_chain              18.687262
front_desk_time_start    99.930772
front_desk_time_end      99.636552
serp_filters             41.683974
star_rating              79.486847
amenity_groups           45.889581
email                    73.014019
images                   62.097612
phone                    71.837141
policy_struct       

In [182]:
# Define the list of columns to check for missing values
columns_to_check = ['address', 'check_in_time', 'check_out_time', 'kind', 'latitude', 'longitude', 'name', 'region', 'metapolicy_struct', 'facts', 'payment_methods', 'hotel_chain']

# Remove rows with missing values in the specified columns
df_cleaned = df.dropna(subset=columns_to_check)

# Output the resulting DataFrame
print(df_cleaned)


                                                 address       check_in_time  \
0                   Via francesco zizzo 28, Санта-Флавия 2023-05-24 23:00:00   
1            Via Cristoforo Colombo; 4, Марина-ди-Модика 2023-05-24 10:00:00   
2        Strada Consortile SantAnna Pollina 131, Альгеро 2023-05-24 20:00:00   
3                            Disseminat 5, 183, Алькудия 2023-05-24 14:00:00   
4               Via Lungomare Luigi Rizzo 131, Лампедуза 2023-05-24 12:00:00   
...                                                  ...                 ...   
18788            Viale Papa Giovanni XXIII 1, Карминьяно 2023-05-24 15:00:00   
18789                               6 Acheloou, Салоники 2023-05-24 16:00:00   
18790      улица Малая Зеленина, д.1/22, Санкт-Петербург 2023-05-24 15:00:00   
18791               улица Николаева, д.18/1, Новосибирск 2023-05-24 14:00:00   
18792  Intersection of Beihang Road and Bizhui Road, ... 2023-05-24 14:00:00   

           check_out_time              

In [183]:
df = df_cleaned.isnull().sum()
print(df)

address                      0
check_in_time                0
check_out_time               0
description_struct          11
id                           0
kind                         0
latitude                     0
longitude                    0
name                         0
postal_code               1484
room_groups               7380
region                       0
deleted                      0
is_closed                    0
metapolicy_struct            0
metapolicy_extra_info     5801
star_certificate         18738
facts                        0
payment_methods              0
hotel_chain                  0
front_desk_time_start    18777
front_desk_time_end      18709
serp_filters              5315
star_rating              14052
amenity_groups            6287
email                    12556
images                   10033
phone                    12284
policy_struct            10839
dtype: int64


In [184]:
display(df_cleaned)

,address,check_in_time,check_out_time,description_struct,id,kind,latitude,longitude,name,postal_code,...,hotel_chain,front_desk_time_start,front_desk_time_end,serp_filters,star_rating,amenity_groups,email,images,phone,policy_struct
0,"Via francesco zizzo 28, Санта-Флавия",2023-05-24 23:00:00,2023-05-24 11:00:00,"[{'paragraphs': ['Прекрасный вариант, чтобы от...",antiche_mura_,Guesthouse,38.089790,13.539411,Antiche Mura,90017,...,No chain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Via Cristoforo Colombo; 4, Марина-ди-Модика",2023-05-24 10:00:00,2023-05-24 12:30:00,"[{'paragraphs': ['Выбор туристов, путешествующ...",amuri_,Apartment,36.712273,14.774748,Amuri,97015,...,No chain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Strada Consortile SantAnna Pollina 131, Альгеро",2023-05-24 20:00:00,2023-05-24 10:00:00,[{'paragraphs': ['Залог хорошего отдыха — чувс...,alghero_country_house,Apartment,40.522330,8.350803,Alghero Country House,07041,...,No chain,None,None,[beach],NaN,NaN,NaN,NaN,NaN,NaN
3,"Disseminat 5, 183, Алькудия",2023-05-24 14:00:00,2023-05-24 12:00:00,"[{'paragraphs': ['Хорошо, когда красота начина...",tia_catalina,Villas_and_Bungalows,39.838790,3.064260,"Son Rotger, villa Tía Catalina con piscina en ...",07400,...,No chain,None,None,NaN,1.0,NaN,NaN,NaN,NaN,NaN
4,"Via Lungomare Luigi Rizzo 131, Лампедуза",2023-05-24 12:00:00,2023-05-24 10:00:00,[{'paragraphs': ['В поисках домашнего уюта: ап...,appartamento_lungomare_2,Apartment,35.498350,12.609085,Appartamento Lungomare,92010,...,No chain,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18788,"Viale Papa Giovanni XXIII 1, Карминьяно",2023-05-24 15:00:00,2023-05-24 10:00:00,[{'paragraphs': ['Бюджетный вариант для активн...,tenuta_di_artimino_appartamenti,Hotel,43.782280,11.043380,Tenuta di Artimino Appartamenti,59015,...,No chain,None,None,"[has_internet, has_airport_transfer, has_parki...",NaN,"[{'amenities': ['Магазины', 'Кондиционер', 'Кр...",villa@artimino.com,[https://cdn.worldota.net/t/{size}/content/33/...,+390558751426,"[{'paragraphs': ['Информация о типе питания, в..."
18789,"6 Acheloou, Салоники",2023-05-24 16:00:00,2023-05-24 11:00:00,"[{'paragraphs': ['Выбор туристов, которые хотя...",ideally_planned_1bedroom_apartment_with_balcon...,Apartment,40.643192,22.914050,Sanders Arch - Cozy 1-bedroom Apartment With B...,546 27,...,No chain,None,None,"[has_internet, has_smoking]",NaN,"[{'amenities': ['Лифт', 'Места для курения'], ...",NaN,[https://cdn.worldota.net/t/{size}/content/26/...,4536950111,"[{'paragraphs': ['Информация о типе питания, в..."
18790,"улица Малая Зеленина, д.1/22, Санкт-Петербург",2023-05-24 15:00:00,2023-05-24 12:00:00,"[{'paragraphs': ['Выбор туристов, путешествующ...",griffin_apart_aristokrat_na_petrogradke_apartm...,Apartment,59.962440,30.282658,Апартаменты GRIFFIN APART Аристократ на Петрог...,NaN,...,No chain,None,None,"[has_internet, has_parking, has_kids, kitchen]",NaN,"[{'amenities': ['Магазины', 'Круглосуточная ст...",katushkafil@yandex.ru,[https://cdn.worldota.net/t/{size}/extranet/0e...,+7 921 926 38 21,"[{'paragraphs': ['Информация о типе питания, в..."
18791,"улица Николаева, д.18/1, Новосибирск",2023-05-24 14:00:00,2023-05-24 12:00:00,[{'paragraphs': ['В поисках домашнего уюта: ап...,welcome_na_nikolaeva_18_apartments,Apartment,54.858223,83.113000,Апартаменты WELCOME на Николаева 18,NaN,...,No chain,None,None,"[has_internet, has_parking, air_conditioning, ...",NaN,"[{'amenities': ['Кондиционер', 'Лифт', 'Отель ...",kvartira.welcome@yandex.ru,[https://cdn.worldota.net/t/{size}/extranet/40...,+7 969 777 62 88,"[{'paragraphs': ['Информация о типе питания, в..."


In [187]:
front_desk_time_start = df_cleaned['front_desk_time_start'].apply(str).unique()
front_desk_time_start = [value for value in front_desk_time_start if value != 'None']
print(front_desk_time_start)

['08:00:00', '09:00:00', '10:00:00', '07:30:00', '07:00:00', '14:00:00', '08:30:00']


In [188]:
front_desk_time_end = df_cleaned['front_desk_time_end'].apply(str).unique()
front_desk_time_end = [value for value in front_desk_time_end if value != 'None']
print(front_desk_time_end)

['23:00:00', '00:00:00', '19:00:00', '21:00:00', '22:00:00', '20:00:00', '17:00:00', '05:30:00', '18:30:00', '20:30:00', '18:00:00', '01:00:00', '22:30:00', '15:00:00', '19:30:00', '02:00:00']


In [189]:
df_cleaned = df_cleaned.drop(['front_desk_time_start', 'front_desk_time_end'], axis=1)


In [190]:
columns_to_replace = ['amenity_groups', 'email', 'images', 'phone', 'policy_struct', 'serp_filters', 'metapolicy_extra_info','room_groups', 'postal_code','description_struct','star_certificate']
df_cleaned[columns_to_replace] = df_cleaned[columns_to_replace].fillna('no information')



In [191]:
star_rating = df_cleaned['star_rating'].apply(str).unique()
star_rating = [value for value in star_rating if value != 'None']
print(star_rating)

['nan', '1.0', '3.0', '2.0', '4.0', '5.0']


In [192]:
df_cleaned['star_rating'] = df_cleaned['star_rating'].fillna('No Rating')


In [193]:
missing_percentages = df_cleaned.isnull().sum() / len(df_cleaned) * 100
print(missing_percentages)

address                  0.0
check_in_time            0.0
check_out_time           0.0
description_struct       0.0
id                       0.0
kind                     0.0
latitude                 0.0
longitude                0.0
name                     0.0
postal_code              0.0
room_groups              0.0
region                   0.0
deleted                  0.0
is_closed                0.0
metapolicy_struct        0.0
metapolicy_extra_info    0.0
star_certificate         0.0
facts                    0.0
payment_methods          0.0
hotel_chain              0.0
serp_filters             0.0
star_rating              0.0
amenity_groups           0.0
email                    0.0
images                   0.0
phone                    0.0
policy_struct            0.0
dtype: float64


In [194]:
duplicates = df_cleaned.duplicated(subset='id')
duplicate_count = duplicates.sum()
print("Number of duplicates:", duplicate_count)


Number of duplicates: 0
